# Import Libraries


In [97]:
import requests #Helps us fetch data from API
import pandas as pd #For handling and analysing data
import numpy as np #For numerical operations
from sklearn.model_selection import train_test_split #Helps to split data into training and testing
from sklearn.preprocessing import LabelEncoder #To convert catogerical data into numerical
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression
from sklearn.metrics import mean_squared_error #to measure accuracy of our prediction
from datetime import datetime,timedelta #to handle date and time
import pytz

In [99]:
API_KEY='6c367973a3bffcc04ddac9e0f9ebc035'
BASE_URL='https://api.openweathermap.org/data/2.5/'


### 1. Fetch Current Weather Data

In [102]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #construct the API request URL
    response=requests.get(url) #send request to API
    data=response.json()
    return{
        'city':data['name'],
        'current_temp':round(data['main']['temp']),
        'feels_like':round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max':round(data['main']['temp_max']),
        'humidity':round(data['main']['humidity']),
        'description':data['weather'][0]['description'],
        'country':data['sys']['country'],
        'wind_gust_dir':data['wind']['deg'],
        'pressure':data['main']['pressure'],
        'Wind_Gust_Speed':data['wind']['speed']
        
            }

### 2. Reading Historical Data

In [105]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    print(f"Original rows: {len(df)}")
    df = df.dropna()
    df = df.drop_duplicates()
    print(f"Cleaned rows: {len(df)}")
    return df

### 3.Prepare data for training

In [108]:
def prepare_data(data):
    le_dir = LabelEncoder()#Label Encoding
    le_rain = LabelEncoder()

    data['WindGustDir'] = le_dir.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le_rain.fit_transform(data['RainTomorrow'])

    x = data.drop('RainTomorrow', axis=1)#feature variable
    y = data['RainTomorrow']#target variable

    return x, y, le_dir

### 4. Train Rain prediction model

In [111]:
def train_rain_model(x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    model=RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    print("Mean Squared Error for Rain Model")
    print(mean_squared_error(y_test,y_pred))
    return model

### 5. Prepare regression data

In [114]:
def prepare_regression_data(data,feature):
    x,y=[],[] #intialize list for feature and target variable
    for i in range(len(data)-1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    x=np.array(x).reshape(-1,1)
    y=np.array(y)
    return x,y
       

## Train Regression Model

In [117]:
def train_regression_model(X,y):
    model=RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(X,y)
    return model

## Predict Future 

In [120]:
def predict_future(model,current_value):
    predictions=[current_value]
    for i in range(5):
        next_value=model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

## Weather Analysis Function

In [123]:
def weather_view():
    city = input("Enter any city name:")
    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = read_historical_data('weather.csv')

    # Prepare and train the rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75), ("N", 348.75, 360)
    ]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression models
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Prepare time for the future prediction
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Prediction")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Prediction")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

# Call the function
weather_view()

Enter any city name: Narasaraopet


Original rows: 366
Cleaned rows: 363
Mean Squared Error for Rain Model
0.1506849315068493
City: Narasaraopet, IN
Current Temperature: 33°C
Feels Like: 35°C
Minimum Temperature: 33°C
Maximum Temperature: 33°C
Humidity: 47%
Rain Prediction: No

Future Temperature Prediction
18:00: 25.1°C
19:00: 20.1°C
20:00: 16.6°C
21:00: 14.4°C
22:00: 13.4°C

Future Humidity Prediction
18:00: 47.2%
19:00: 47.2%
20:00: 47.2%
21:00: 47.2%
22:00: 47.2%
